In [8]:
# Importar las librerías necesarias
import torch
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import os

# Verificar la disponibilidad de la GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Directorio para guardar las imágenes generadas
OUTPUT_DIR = "ecommerce_generated_images"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Configurar el modelo y el scheduler
MODEL_ID = "stabilityai/stable-diffusion-2-1-base"

# Cargar el scheduler
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")

# Configurar el pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    scheduler=scheduler,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

# Desactivar el safety checker para evitar bloqueos (opcional)
pipe.safety_checker = None

# Generar imágenes a partir de prompts
def generate_image(prompt, guidance_scale=7.5, num_inference_steps=50, file_name="generated_image.png"):
    print(f"Generating image for prompt: {prompt}")
    image = pipe(prompt, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images[0]
    file_path = os.path.join(OUTPUT_DIR, file_name)
    image.save(file_path)
    print(f"Image saved to: {file_path}")
    return image

# Prompts para las imágenes
prompts = [
    "A high-resolution product photo of modern running sneakers on a white background",
    "A high-resolution product photo of a luxury leather handbag with gold accents, studio lighting, white background"
]

# Generar imágenes para cada prompt
for idx, prompt in enumerate(prompts, start=1):
    generate_image(prompt, file_name=f"product_{idx}.png")

# Image-to-Image: Variación de una imagen base
from io import BytesIO
import requests

# Cargar una imagen base desde URL
base_image_url = "https://example.com/path_to_image.jpg"  # Reemplaza con una URL válida
response = requests.get(base_image_url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image = init_image.resize((512, 512))

# Guardar la imagen base
init_image.save(os.path.join(OUTPUT_DIR, "base_image.png"))
print("Base image saved.")

# Generar variaciones de la imagen base
def generate_image_variation(init_image, prompt, strength=0.7, file_name="variation_image.png"):
    print(f"Generating variation for prompt: {prompt}")
    image = pipe(prompt=prompt, init_image=init_image, strength=strength, guidance_scale=8.0).images[0]
    file_path = os.path.join(OUTPUT_DIR, file_name)
    image.save(file_path)
    print(f"Variation image saved to: {file_path}")
    return image

# Prompt para la variación
variation_prompt = "A futuristic version of the product with neon accents and a dark background"

# Generar y guardar la variación
generate_image_variation(init_image, variation_prompt, file_name="variation_image.png")



Using device: cpu


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  9.05it/s]


Generating image for prompt: A high-resolution product photo of modern running sneakers on a white background


100%|██████████| 50/50 [08:02<00:00,  9.65s/it]


Image saved to: ecommerce_generated_images\product_1.png
Generating image for prompt: A high-resolution product photo of a luxury leather handbag with gold accents, studio lighting, white background


100%|██████████| 50/50 [08:06<00:00,  9.72s/it]


Image saved to: ecommerce_generated_images\product_2.png


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001600016E2A0>